# Workshop 1 - Apply
    
In this notebook you will work on a homework assignment involving a Vierendeel frame.

Our matrix method implementation is now completely stored in a local package, consisting of three classes.

## Vierendeel frame

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/vierendeel.png" WIDTH=400 ALIGN="center">
      </figure></center>

With:

- $h = 1$
- $b = 1$
- $EI_r = 10000$
- $EI_k = 1000$
- $EA  = 1\cdot 10^{10}$
- $H = 100$


In the first half of this course last quarter, you have learned that the deformation of Vierendeel frames (an example of which is shown above) can be obtained in a simplified way by assuming the global deformation can be described by a shear beam with equivalent stiffness given by:

$$
k = \frac{24}{h\left(\displaystyle\frac{h}{EI_k}+\frac{b}{EI_r}\right)}
$$

Now that you have the tools to solve the original frame problem using the Matrix Method, your task in this assignment is to investigate the validity of this equivalent shear beam model.

Note that the checks only had a single element. For this model you need to obtain $\mathbf{K}$ and $\mathbf{f}$ of all elements and add them to the correct locations in the global stiffness matrix and force vector. To do that, make use of the `global_dofs` function of the Element class and the `np.ix_` Numpy utility function. (Tip: refer back to what you did in the `constrain` function).

Once you have a solution, use SymPy / Maple / pen and paper to solve a shear beam problem with the equivalent stiffness given above (It is very similar to the simple extension problem above) and compare the horizontal displacement at the point of application of $H$ for the two models.

Investigate how the two models compare for different values of $EA$, ranging from very small (*e.g.* $1\cdot 10^{-5}$) to very large (*e.g.* $1\cdot10^{10}$). What explains the behavior you observe?

In [100]:
import numpy as np
import matrixmethod as mm
%config InlineBackend.figure_formats = ['svg']

In [101]:
mm.Node.clear()
mm.Element.clear()

In [102]:
#YOUR CODE HERE
node1 = mm.Node(0, 0)
node2 = mm.Node(1, 0)
node3 = mm.Node(0, -1)
node4 = mm.Node(1, -1)
nodes = [node1, node2, node3, node4]
elem1 = mm.Element(node1, node2)
elem2 = mm.Element(node4, node2)
elem3 = mm.Element(node3, node4)
elem4 = mm.Element(node3, node1)
elems = [elem1, elem2, elem3, elem4]
section_r = {}
section_r['EI'] = 10000
section_k = {}
section_k['EI'] = 1000
section_k['EA'] = section_r['EA'] = 10e10
elems[0].set_section(section_r)
elems[2].set_section(section_r)
elems[1].set_section(section_k)
elems[3].set_section(section_k)

In [103]:
global_k = np.zeros((len(nodes)*3, len(nodes)*3))
global_f = np.zeros(len(nodes)*3)
H = 100
global_f[8] = H
for elem in elems:
    elmat = elem.stiffness()
    idofs = elem.global_dofs()
    global_k[np.ix_(idofs, idofs)] += elmat
print(f'The Shape of Global matrix is {global_k.shape}')

con = mm.Constrainer()
for node in nodes:
    if node in [nodes[0], nodes[1]]:
        con.fix_dof(node, 0)
        con.fix_dof(node, 1)
print(con)
node3.add_load([H, 0, 0])

The Shape of Global matrix is (12, 12)
This constrainer has constrained the degrees of freedom: [0, 1, 3, 4] with corresponding constrained values: [0, 0, 0, 0])


In [104]:
Kff , Ff = con.constrain(global_k, global_f)
print(f'The Shape of reduced matrix is {Kff.shape}')
u = np.linalg.solve(Kff, Ff)
print(f' u = {u}')
print(f'The top-left horizontal displacement is {u[2]}')

The Shape of reduced matrix is (8, 8)
 u = [-1.61384372e-04  1.88266112e-04  4.16667025e-04  9.83871416e-10
  2.91779614e-03  4.16666909e-04  7.66806268e-10 -1.27801001e-03]
The top-left horizontal displacement is 0.00041666702467128456


In [ ]:
#provided in case you want to solve the shear beam problem using SymPy
import sympy as sym
x, k, L, H = sym.symbols('x, k, L, H')
w = sym.Function('w')

ODE_shear = sym.Eq(sym.diff(w(x), x, x), 0)
ODE_shear_sol = sym.dsolve(ODE_shear)
print(ODE_shear_sol)
boundary_conditions = [sym.Eq(w(0), 0), sym.Eq(k * sym.diff(w(x), x).subs(x, L), H)]
constants = sym.solve(boundary_conditions)
ODE_shear_sol = ODE_shear_sol.subs(constants)


Eq(w(x), C1 + C2*x)
Eq(w(x), C1 + C2*x)
C1 + C2*x
C1 + C2*x
C1 + C2*x
